In [1]:
import pandas as pd
import os
import glob
import matplotlib.pyplot as plt
import matplotlib.patches as patches
import math
from scipy.spatial import ConvexHull, convex_hull_plot_2d
import imageio
import numpy as np
import warnings
import torch
from torch import nn
import torch.optim as opt

In [2]:
from tensorflow.keras import backend as K
from tensorflow.keras.models import load_model

model = load_model('model_full_fold.h5')

In [38]:
def get_tracking_and_plays(filepath):
    """
    Return a dataframe containing cleaned tracking data joined with the plays data
    """
    tracking = pd.read_csv(filepath)
    plays = pd.read_csv("plays.csv")
    tracking = tracking[tracking['playId'].isin(tracking[tracking['event'] != 'fumble']['playId'].unique())]
    plays = plays[plays['playNullifiedByPenalty'] == 'N']

    tracking.loc[tracking['playDirection'] == 'left', 'x'] = 120 - tracking.loc[tracking['playDirection'] == 'left', 'x']
    tracking.loc[tracking['playDirection'] == 'left', 'y'] = (160/3) - tracking.loc[tracking['playDirection'] == 'left', 'y']
    tracking.loc[tracking['playDirection'] == 'left', 'dir'] += 180
    tracking.loc[tracking['dir'] > 360, 'dir'] -= 360
    tracking.loc[tracking['playDirection'] == 'left', 'o'] += 180
    tracking.loc[tracking['o'] > 360, 'o'] -= 360

    tracking_with_plays = tracking.merge(plays, on=['gameId', 'playId'], how='left')

    tracking_with_plays['is_on_offense'] = tracking_with_plays['club'] == tracking_with_plays['possessionTeam']
    tracking_with_plays['is_on_defence'] = tracking_with_plays['club'] == tracking_with_plays['defensiveTeam']
    tracking_with_plays['is_ballcarrier'] = tracking_with_plays['ballCarrierId'] == tracking_with_plays['nflId']

    football_coords=tracking_with_plays.loc[tracking_with_plays['displayName']=='football']
    football_coords['football_x']=football_coords['x']
    football_coords['football_y']=football_coords['y']
    football_coords=football_coords[['gameId', 'playId', 'frameId', 'football_x', 'football_y']]

    tracking_with_plays=tracking_with_plays.merge(football_coords, on=['gameId', 'playId', 'frameId'], how='left')
    tracking_with_plays['dir_football']=(np.degrees(np.arctan2(tracking_with_plays['y'] - tracking_with_plays['football_y'], tracking_with_plays['x'] - tracking_with_plays['football_x']))+360)%360
    tracking_with_plays['dir_player']=tracking_with_plays['dir']

    end_frame = tracking_with_plays[tracking_with_plays['event'].isin(['tackle', 'out_of_bounds'])].groupby('playId')['frameId'].min().reset_index()
    end_frame.rename(columns={'frameId': 'frameId_end'}, inplace=True)

    start_frame = tracking_with_plays[tracking_with_plays['event'].isin(['run', 'lateral', 'run_pass_option', 'handoff', 'pass_arrived'])].groupby('playId')['frameId'].min().reset_index()
    start_frame.rename(columns={'frameId': 'frameId_start'}, inplace=True)

    tracking_with_plays = tracking_with_plays.merge(end_frame, on='playId', how='left')
    tracking_with_plays = tracking_with_plays.merge(start_frame, on='playId', how='left')

    tracking_with_plays = tracking_with_plays[(tracking_with_plays['frameId'] <= tracking_with_plays['frameId_end']) &
                                              (tracking_with_plays['frameId'] >= tracking_with_plays['frameId_start'])]

    return tracking_with_plays

In [80]:
tracking_with_plays = get_tracking_and_plays('tracking_week_3.csv')

tracking_with_plays.loc[(tracking_with_plays['gameId']==2022092200) & (tracking_with_plays['playId']==3093) & (tracking_with_plays['frameId']==25)]

,gameId,playId,nflId,displayName,frameId,time,jerseyNumber,club,playDirection,x,...,foulNFLId2,is_on_offense,is_on_defence,is_ballcarrier,football_x,football_y,dir_football,dir_player,frameId_end,frameId_start
81030,2022092200,3093,35449.0,Tyson Alualu,25,2022-09-22 22:35:19.200000,94.0,PIT,right,72.250000,...,NaN,False,True,False,67.529999,28.24,356.605080,4.64,84.0,20.0
81118,2022092200,3093,41264.0,Joel Bitonio,25,2022-09-22 22:35:19.200000,75.0,CLE,right,71.080000,...,NaN,True,False,False,67.529999,28.24,14.830915,357.30,84.0,20.0
81206,2022092200,3093,43297.0,Jack Conklin,25,2022-09-22 22:35:19.200000,78.0,CLE,right,70.430000,...,NaN,True,False,False,67.529999,28.24,322.317022,9.27,84.0,20.0
81294,2022092200,3093,43325.0,Myles Jack,25,2022-09-22 22:35:19.200000,51.0,PIT,right,73.650000,...,NaN,False,True,False,67.529999,28.24,349.357740,358.39,84.0,20.0
81382,2022092200,3093,43380.0,Jacoby Brissett,25,2022-09-22 22:35:19.200000,7.0,CLE,right,64.930000,...,NaN,True,False,False,67.529999,28.24,228.895859,247.81,84.0,20.0
81470,2022092200,3093,44841.0,David Njoku,25,2022-09-22 22:35:19.200000,85.0,CLE,right,70.780000,...,NaN,True,False,False,67.529999,28.24,294.443964,338.71,84.0,20.0
81558,2022092200,3093,44870.0,Ethan Pocic,25,2022-09-22 22:35:19.200000,55.0,CLE,right,72.470000,...,NaN,True,False,False,67.529999,28.24,6.925083,27.43,84.0,20.0
81646,2022092200,3093,44877.0,Larry Ogunjobi,25,2022-09-22 22:35:19.200000,99.0,PIT,right,70.530000,...,NaN,False,True,False,67.529999,28.24,310.601308,342.97,84.0,20.0
81734,2022092200,3093,44886.0,Chris Wormley,25,2022-09-22 22:35:19.200000,95.0,PIT,right,71.640000,...,NaN,False,True,False,67.529999,28.24,22.352080,353.90,84.0,20.0
81822,2022092200,3093,44906.0,Cameron Sutton,25,2022-09-22 22:35:19.200000,20.0,PIT,right,80.970000,...,NaN,False,True,False,67.529999,28.24,49.162857,83.47,84.0,20.0


In [5]:
def create_feature_tensor(feature_df):
    """
    Convert the input frame_df to a 4D tensor.
        - The first dimension is the frame
        - The second dimension is the index of the current player
        - The third dimension is the index of the relative player
    """
    tensor_shape = (feature_df.groupby(['gameId', 'playId', 'frameId']).ngroups, 10, 11, 10)
    input_tensor = np.zeros(tensor_shape)
    cur_count = 0
    for game_id, game_group in feature_df.groupby('gameId'):
        for play_id, play_group in game_group.groupby('playId'):
            for frame_id, frame_group in play_group.groupby('frameId'):
                offense = frame_group[(frame_group['is_on_offense']) & (~frame_group['is_ballcarrier'])]
                defence = frame_group[frame_group['is_on_defence']]
                ballcarrier = frame_group[frame_group['is_ballcarrier']]
                ballcarrier_sx = ballcarrier.s * np.cos(np.deg2rad(ballcarrier.dir))
                ballcarrier_sy = ballcarrier.s * np.sin(np.deg2rad(ballcarrier.dir))
                for i, def_player in enumerate(defence.itertuples()):
                    def_player_sx = def_player.s * np.cos(np.deg2rad(def_player.dir))
                    def_player_sy = def_player.s * np.sin(np.deg2rad(def_player.dir))
                    for j, off_player in enumerate(offense.itertuples()):
                        off_player_sx = off_player.s * np.cos(np.deg2rad(off_player.dir))
                        off_player_sy = off_player.s * np.sin(np.deg2rad(off_player.dir))
                        input_tensor[cur_count, 0, i, j] = off_player.x - def_player.x
                        input_tensor[cur_count, 1, i, j] = off_player.y - def_player.y
                        input_tensor[cur_count, 2, i, j] = def_player_sx
                        input_tensor[cur_count, 3, i, j] = def_player_sy
                        input_tensor[cur_count, 4, i, j] = def_player_sx - ballcarrier_sx
                        input_tensor[cur_count, 5, i, j] = def_player_sy - ballcarrier_sy
                        input_tensor[cur_count, 6, i, j] = def_player.x - ballcarrier.x
                        input_tensor[cur_count, 7, i, j] = def_player.y - ballcarrier.y
                        input_tensor[cur_count, 8, i, j] = off_player_sx - def_player_sx
                        input_tensor[cur_count, 9, i, j] = off_player_sy - def_player_sy
                cur_count += 1
    return input_tensor

Counterfactual Analysis

In [87]:
import numpy as np

def change_distance(df, displayName, amount):
    df['sx'] = df['s'] * np.cos(np.deg2rad(df['dir_football']))
    df['sy'] = df['s'] * np.sin(np.deg2rad(df['dir_football']))

    filtered_df = df[df['displayName'] == displayName].copy()
    
    filtered_df['px'] = filtered_df['x'].shift(1)
    filtered_df['pxs'] = filtered_df['sx'].shift(1)
    filtered_df['py'] = filtered_df['y'].shift(1)
    filtered_df['psy'] = filtered_df['sy'].shift(1)

    for index, row in filtered_df.iterrows():
        if pd.isna(row['px']) or pd.isna(row['pxs']) or pd.isna(row['py']) or pd.isna(row['psy']):
            continue
        
        # print(df.at[index, 'x'])
        df.at[index, 'sx'] = row['pxs']
        df.at[index, 'sy'] = row['psy']
        df.at[index, 'x'] = row['px'] + amount * row['pxs']
        df.at[index, 'y'] = row['py'] + amount * row['psy']
        # print(df.at[index, 'x'])
    return df

In [56]:
def compare_yard_prediction(tracking_with_plays, displayName, gameId, playId, amount):
  play_info = tracking_with_plays.loc[(tracking_with_plays['playId'] == playId) & (tracking_with_plays['gameId'] == gameId)]
  original_prediction = model.predict(create_feature_tensor(play_info))
  changed_play_info = change_distance(play_info, displayName, amount)
  changed_prediction = model.predict(create_feature_tensor(changed_play_info))
  prediction_difference = changed_prediction - original_prediction
  results = pd.DataFrame({
        'Original Prediction': original_prediction.flatten(),
        'Changed Prediction': changed_prediction.flatten(),
        'Prediction Difference': prediction_difference.flatten()
    })
  print(1)
  return results

In [93]:
players=tracking_with_plays.loc[(tracking_with_plays['gameId']==2022092200) & (tracking_with_plays['playId']==3114) & (tracking_with_plays['is_on_defence']==True)]['displayName'].unique()
for i in players:
    df=compare_yard_prediction(tracking_with_plays, i, 2022092200, 3114, 0.1)
    break

df

2/2 [==============================] - 0s 2ms/step
1


,Original Prediction,Changed Prediction,Prediction Difference
0,5.645452,5.645452,0.000000
1,5.457328,5.466942,0.009614
2,4.989746,4.992747,0.003001
3,4.456761,4.461836,0.005075
4,4.194693,4.198458,0.003765
5,4.246519,4.247785,0.001266
6,4.272907,4.270060,-0.002847
7,4.611137,4.610578,-0.000559
8,4.865740,4.861642,-0.004098
9,4.868312,4.862154,-0.006158


In [9]:
plays=pd.read_csv('plays.csv')
games=pd.read_csv('games.csv')
tracking1 = get_tracking_and_plays('tracking_week_1.csv')
tracking2 = get_tracking_and_plays('tracking_week_2.csv')
tracking3 = get_tracking_and_plays('tracking_week_3.csv')
tracking4 = get_tracking_and_plays('tracking_week_4.csv')
tracking5 = get_tracking_and_plays('tracking_week_5.csv')
tracking6 = get_tracking_and_plays('tracking_week_6.csv')
tracking7 = get_tracking_and_plays('tracking_week_7.csv')
tracking8 = get_tracking_and_plays('tracking_week_8.csv')
tracking9 = get_tracking_and_plays('tracking_week_9.csv')

In [10]:
def get_tracking(week_num):
    if week_num==1:
        return tracking1
    elif week_num==2:
        return tracking2
    elif week_num==3:
        return tracking3
    elif week_num==4:
        return tracking4
    elif week_num==5:
        return tracking5
    elif week_num==6:
        return tracking6
    elif week_num==7:
        return tracking7
    elif week_num==8:
        return tracking8
    else:
        return tracking9

In [179]:
warnings.filterwarnings('ignore')
distinct_plays=plays.sort_values(by=['gameId','playId'])[['gameId', 'playId']]
results_dict={}

counter=0
for i, row in distinct_plays.iterrows():
    week_number=int(games.loc[games['gameId']==row['gameId']]['week'])
    tracking=get_tracking(week_number)
    players=tracking.loc[(tracking['gameId']==row['gameId']) & (tracking['playId']==row['playId']) & (tracking['is_on_defence']==True)]['displayName'].unique()
    for j in players:
        final_df=compare_yard_prediction(tracking, j, int(row['gameId']), int(row['playId']), 0.1)
        cumulative_impact=sum(final_df['Prediction Difference'])
        if j in results_dict:
            results_dict[j]+=cumulative_impact
        else:
            results_dict[j]=cumulative_impact
    counter+=1
    if counter>10:
        break

1/1 [==============================] - 0s 10ms/step
1
17
1/1 [==============================] - 0s 9ms/step
1
17
1/1 [==============================] - 0s 11ms/step
1
17
1/1 [==============================] - 0s 12ms/step
1
17
1/1 [==============================] - 0s 12ms/step
1
17
1/1 [==============================] - 0s 10ms/step


ValueError: setting an array element with a sequence.

Difference between Expected Yards and Realized Yards at Start of Play

In [11]:
def get_prediction_difference(tracking_with_plays, gameId, playId):
  play_info = tracking_with_plays.loc[(tracking_with_plays['playId'] == playId) & (tracking_with_plays['gameId'] == gameId)]
  first_coord_football=float(play_info[(play_info['frameId']==play_info['frameId_start']) & (play_info['displayName']=='football')]['x'])
  last_coord_football=float(play_info[(play_info['frameId']==play_info['frameId_end']) & (play_info['displayName']=='football')]['x'])
  original_prediction = model.predict(create_feature_tensor(play_info))
  actual_yards=last_coord_football-first_coord_football
  ball_carrier=play_info.iloc[0]['ballCarrierDisplayName']
  defense_name=play_info.iloc[0]['defensiveTeam']
  return actual_yards-original_prediction[0][0], ball_carrier, defense_name

In [12]:
get_prediction_difference(tracking1, 2022090800, 818)

/var/folders/pq/qr32tkk90n7361gslj40w8j4tyz0cb/T/ipykernel_82194/2488794438.py:3: FutureWarning: Calling float on a single element Series is deprecated and will raise a TypeError in the future. Use float(ser.iloc[0]) instead
  first_coord_football=float(play_info[(play_info['frameId']==play_info['frameId_start']) & (play_info['displayName']=='football')]['x'])
/var/folders/pq/qr32tkk90n7361gslj40w8j4tyz0cb/T/ipykernel_82194/2488794438.py:4: FutureWarning: Calling float on a single element Series is deprecated and will raise a TypeError in the future. Use float(ser.iloc[0]) instead
  last_coord_football=float(play_info[(play_info['frameId']==play_info['frameId_end']) & (play_info['displayName']=='football')]['x'])
/var/folders/pq/qr32tkk90n7361gslj40w8j4tyz0cb/T/ipykernel_82194/1026185271.py:29: FutureWarning: Calling float on a single element Series is deprecated and will raise a TypeError in the future. Use float(ser.iloc[0]) instead
  input_tensor[cur_count, 4, i, j] = def_player_sx 

1/1 [==============================] - 0s 15ms/step


(1.9375271797180602, 'Darrell Henderson', 'BUF')

In [30]:
warnings.filterwarnings('ignore')
distinct_plays=plays.sort_values(by=['gameId','playId'])[['gameId', 'playId']]
team_results={}

games=games.sort_values(by='week')

counter=0
for i, row in distinct_plays.iterrows():
    week_number=int(games.loc[games['gameId']==row['gameId']]['week'])
    tracking=get_tracking(week_number)
    try:
        tuple_vals=get_prediction_difference(tracking, row['gameId'], row['playId'])
        if tuple_vals[1] in team_results:
         team_results[tuple_vals[1]]+=tuple_vals[0]
        else:
            team_results[tuple_vals[1]]=tuple_vals[0]
        if tuple_vals[2] in team_results:
            team_results[tuple_vals[2]]-=tuple_vals[0]
        else:
            team_results[tuple_vals[2]]=tuple_vals[0]*-1
    except:
        print(row['gameId'], row['playId'])
    counter+=1

team_results

1/1 [==============================] - 0s 9ms/step
2022100201 3542
1/1 [==============================] - 0s 10ms/step
2022100202 709
1/1 [==============================] - 0s 9ms/step
2022100202 881
2/2 [==============================] - 0s 2ms/step
2022100202 1009
2/2 [==============================] - 0s 2ms/step
2022100202 1293
2/2 [==============================] - 0s 2ms/step
2022100202 1369
1/1 [==============================] - 0s 10ms/step
2022100202 1433
1/1 [==============================] - 0s 9ms/step
2022100202 2019
1/1 [==============================] - 0s 9ms/step
2022100202 2373
1/1 [==============================] - 0s 9ms/step
2022100202 2574
2/2 [==============================] - 0s 2ms/step
2022100202 3268
2/2 [==============================] - 0s 2ms/step
2022100202 3483
1/1 [==============================] - 0s 9ms/step
2022100202 3621
2/2 [==============================] - 0s 1ms/step
2022100203 2440
1/1 [==============================] - 0s 10ms/step
2022100203

{'Stefon Diggs': 76.70538434386184,
 'LA': -339.78115597367446,
 'Josh Allen': 87.92232686281224,
 'Devin Singletary': 86.04957437515299,
 'Zack Moss': 68.79053702950544,
 'Darrell Henderson': -14.729316651819747,
 'BUF': -453.1729350984132,
 'Cooper Kupp': 44.876804918051114,
 'Tyler Higbee': 64.56052201986364,
 'Ben Skowronek': 37.36804196238532,
 'Jamison Crowder': 2.415125489235052,
 'Isaiah McKenzie': 6.111850976943934,
 'Dawson Knox': -26.23320931196196,
 'Cam Akers': -79.77077853679748,
 'Gabe Davis': 11.16139400005316,
 'Brandon Powell': 40.243643820286955,
 'Allen Robinson': -20.70005300641074,
 'Matthew Stafford': -3.0891325473784974,
 'Marcus Mariota': 123.30519008636465,
 'NO': -140.67162051796657,
 'Cordarrelle Patterson': 23.24673151970007,
 'Parker Hesse': 13.075969815255043,
 'Damien Williams': -2.1463484764098766,
 'Kyle Pitts': 59.74632185697537,
 'Alvin Kamara': 29.95114758610742,
 'ATL': -244.07688891887352,
 'Jameis Winston': -3.116720199585018,
 'Olamide Zaccheaus

In [35]:
import operator
sorted_d = sorted(team_results.items(), key=operator.itemgetter(1), reverse=True)
sorted_d

[('Lamar Jackson', 439.4007861018178),
 ('Travis Etienne', 285.69930595159553),
 ('Justin Fields', 279.5212503373616),
 ('Miles Sanders', 274.7611335515975),
 ('Rhamondre Stevenson', 223.9665178358567),
 ('Khalil Herbert', 211.95844608545283),
 ('Kyler Murray', 184.53632259368902),
 ('Saquon Barkley', 182.89138776064118),
 ('Dameon Pierce', 165.26517754793426),
 ('Aaron Jones', 162.49561327695807),
 ('Justin Jefferson', 160.74112063646353),
 ('Kenyan Drake', 157.46185284853),
 ('Austin Ekeler', 151.79632234573302),
 ('Dallas Goedert', 143.32377177476965),
 ('Terry McLaurin', 137.83208358287843),
 ('Tyler Allgeier', 137.61533141136047),
 ('Christian McCaffrey', 137.3318694233894),
 ('David Montgomery', 135.41521602869113),
 ('Deebo Samuel', 133.47172492742573),
 ('Joe Mixon', 127.55844670534117),
 ('David Njoku', 127.48708564043018),
 ('Jaylen Waddle', 125.41608536243342),
 ('Marcus Mariota', 123.30519008636465),
 ('Jaylen Warren', 120.0756590068334),
 ('James Conner', 118.8274457454697

Tackle Evasion Analysis

In [13]:
def get_broken_tackles(tracking_with_plays, gameId, playId):
  play_info = tracking_with_plays.loc[(tracking_with_plays['playId'] == playId) & (tracking_with_plays['gameId'] == gameId)].reset_index()
  original_prediction = model.predict(create_feature_tensor(play_info))
  index=np.argmax(original_prediction<1)
  tackle_evasion=0
  tackle_evaded_yards=0
  ball_carrier=play_info.iloc[0]['ballCarrierDisplayName']
  defense_name=play_info.iloc[0]['defensiveTeam']
  if index>0:
    first_coord_football=float(play_info[(play_info['displayName']=='football')].iloc[index]['x'])
    last_coord_football=float(play_info[(play_info['frameId']==play_info['frameId_end']) & (play_info['displayName']=='football')]['x'])
    if (last_coord_football-first_coord_football)>5:
      tackle_evasion=1
      tackle_evaded_yards=last_coord_football-first_coord_football 
  return tackle_evasion, tackle_evaded_yards, ball_carrier, defense_name

In [26]:
warnings.filterwarnings('ignore')
distinct_plays=plays.sort_values(by=['gameId','playId'], ascending=False)[['gameId', 'playId']]
ball_carrier_results_tackles_evaded={}
ball_carrier_results_tackes_yards={}
defense_results_tackles_evaded={}
defense_results_tackles_yards={}

games=games.sort_values(by='week')

counter=0
for i, row in distinct_plays.iterrows():
    week_number=int(games.loc[games['gameId']==row['gameId']]['week'])
    tracking=get_tracking(week_number)
    try:
        tuple_vals=get_broken_tackles(tracking, row['gameId'], row['playId'])
        if tuple_vals[2] in ball_carrier_results_tackles_evaded:
            ball_carrier_results_tackles_evaded[tuple_vals[2]]+=tuple_vals[0]
            ball_carrier_results_tackes_yards[tuple_vals[2]]+=tuple_vals[1]
        else:
            ball_carrier_results_tackles_evaded[tuple_vals[2]]=tuple_vals[0]
            ball_carrier_results_tackes_yards[tuple_vals[2]]=tuple_vals[1]
        if tuple_vals[3] in defense_results_tackles_evaded:
            defense_results_tackles_evaded[tuple_vals[3]]-=tuple_vals[0]
            defense_results_tackles_yards[tuple_vals[3]]-=tuple_vals[1]
        else:
            defense_results_tackles_evaded[tuple_vals[3]]=tuple_vals[0]
            defense_results_tackles_yards[tuple_vals[3]]=tuple_vals[1]
    except:
        print(row['gameId'], row['playId'])
    counter+=1

print(ball_carrier_results_tackles_evaded)
print(ball_carrier_results_tackes_yards)
print(defense_results_tackles_evaded)
print(defense_results_tackles_yards)

1/1 [==============================] - 0s 10ms/step
2022110700 3480
2/2 [==============================] - 0s 2ms/step
2022110700 3401
1/1 [==============================] - 0s 10ms/step
2022110700 3195
1/1 [==============================] - 0s 10ms/step
2022110700 2496
1/1 [==============================] - 0s 10ms/step
2022110700 2412
2/2 [==============================] - 0s 1ms/step
2022110700 2182
1/1 [==============================] - 0s 9ms/step
2022110700 1768
2022110700 1734
1/1 [==============================] - 0s 10ms/step
2022110700 839
2/2 [==============================] - 0s 2ms/step
2022110700 700
2/2 [==============================] - 0s 2ms/step
2022110700 646
1/1 [==============================] - 0s 11ms/step
2022110700 518
1/1 [==============================] - 0s 10ms/step
2022110610 4348
2/2 [==============================] - 0s 2ms/step
2022110610 4162
1/1 [==============================] - 0s 9ms/step
2022110610 3836
1/1 [==============================] - 0s 9

In [28]:
import operator
sorted_d = sorted(ball_carrier_results_tackes_yards.items(), key=operator.itemgetter(1), reverse=True)
sorted_d

[('Khalil Herbert', 144.67999458312968),
 ('Derrick Henry', 132.4299964904784),
 ('Josh Jacobs', 132.11999320983878),
 ('Rhamondre Stevenson', 125.42999935150169),
 ('Dameon Pierce', 108.57999801635808),
 ('Saquon Barkley', 98.88999938964838),
 ('David Montgomery', 98.46000671386682),
 ('Nick Chubb', 92.38000679016136),
 ('Travis Etienne', 87.6899986267092),
 ('Lamar Jackson', 85.39999866485608),
 ('Kareem Hunt', 84.66000175476067),
 ('Leonard Fournette', 75.17999649047823),
 ('Kenyan Drake', 71.18999290466297),
 ('Austin Ekeler', 67.5400047302246),
 ('Eno Benjamin', 66.64000129699708),
 ('Justin Fields', 66.15000343322768),
 ('Miles Sanders', 62.92999649047841),
 ('Breece Hall', 62.1799945831297),
 ('Jamaal Williams', 61.9200038909911),
 ('Kevin White', 60.8800010681148),
 ('Dallas Goedert', 60.73000907897954),
 ('Kenneth Walker', 58.22999954223641),
 ('Christian McCaffrey', 56.609996795654325),
 ('Michael Carter', 52.3700065612793),
 ('Brian Robinson', 51.72000217437739),
 ('Devin Si

In [152]:
tracking3.loc[(tracking3['playId'] == 419)& (tracking3['gameId']==2022092200)]['gameClock']

6388    8:37
6389    8:37
6390    8:37
6391    8:37
6392    8:37
        ... 
8133    8:37
8134    8:37
8135    8:37
8136    8:37
8137    8:37
Name: gameClock, Length: 1288, dtype: object

In [ ]:
warnings.filterwarnings('ignore')
distinct_plays=plays.sort_values(by=['gameId','playId'])[['gameId', 'playId']]
player_results_broken_tackles={}
player_results_expected_yards={}

games=games.sort_values(by='week')

counter=0
for i, row in distinct_plays.iterrows():
    week_number=int(games.loc[games['gameId']==row['gameId']]['week'])
    tracking=get_tracking(week_number)
    try:
        tuple_vals=get_prediction_difference(tracking, row['gameId'], row['playId'])
        if tuple_vals[1] in team_results:
         player_results[tuple_vals[1]]+=tuple_vals[0]
        else:
            team_results[tuple_vals[1]]=tuple_vals[0]
        if tuple_vals[2] in team_results:
            team_results[tuple_vals[2]]-=tuple_vals[0]
        else:
            team_results[tuple_vals[2]]=tuple_vals[0]*-1
    except:
        print(row['gameId'], row['playId'])
    counter+=1

team_results